Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 3.66 ms, sys: 297 µs, total: 3.96 ms
Wall time: 11.4 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.1.0.145/6601/1 Dashboard: http://10.1.0.145:8787/status,Cluster Workers: 1 Cores: 6 Memory: 7.29 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 219 ms, sys: 27.9 ms, total: 247 ms
Wall time: 2.17 s


[1.7790483514797581,
 1.9862238839181945,
 1.7425224076767525,
 1.5307340938936982,
 1.3052023901923906,
 1.3099432078682964,
 1.7001737236485224,
 1.7509681966534898,
 1.3408999445388736,
 1.707382440128397,
 1.857263774634613,
 1.9873704220569157,
 1.9594656299565543,
 1.756877587433907,
 1.2950997727357734,
 1.8928501604846084,
 1.8988149121485192,
 1.6455622462248272,
 1.5452257498197088,
 1.5080837975992614,
 1.9240814314539536,
 1.0868906443944222,
 1.08407249225558,
 1.983393230661565,
 1.4347283580482042,
 1.6840951859299629,
 1.192261559315308,
 1.2235966339817963,
 1.0291084223212614,
 1.8988547853101294,
 1.1668057434754124,
 1.546682205687529,
 1.2340587374238436,
 1.4804258729296027,
 1.4866906883276276,
 1.3215611337799653,
 1.3851574002015812,
 1.4249096273840371,
 1.6363502568965949,
 1.308890773141898,
 1.685614015411215,
 1.9235382924371602,
 1.7283059048520562,
 1.7073696518901404,
 1.788933484976849,
 1.4496667216415835,
 1.6681644475787794,
 1.6671595766646403,
 1.

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.9010679368350707,
 1.2430871056099635,
 1.9603716674787948,
 1.2498039558802712,
 1.4463221737038374,
 1.2883313633842275,
 1.5844010566681939,
 1.8829638891253309,
 1.2301620367572539,
 1.9003620450089764,
 1.0617419252491285,
 1.9357287705338657,
 1.9498484902776445,
 1.928274670024439,
 1.714467712117714,
 1.253794846043224,
 1.7314376910824432,
 1.910421081821925,
 1.1340506175226732,
 1.2830354429350619,
 1.8904411066111648,
 1.2833422983762532,
 1.3755923323071864,
 1.0231901046664493,
 1.4974800942892839,
 1.9866535919149095,
 1.899066004820728,
 1.9777075676114309,
 1.270014974969983,
 1.7373747391606327,
 1.7215033312131953,
 1.2618939714346085,
 1.948310434819149,
 1.153148842901111,
 1.1213496993558056,
 1.3974727500796815,
 1.9196781191567478,
 1.8527486226041043]

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 203 ms, sys: 7.21 ms, total: 210 ms
Wall time: 2.41 s
